## Make PCTS data tract level

In [1]:
import intake
import geopandas as gpd
import numpy as np
import pandas as pd

In [2]:
catalog = intake.open_catalog("../catalogs/*.yml")

bucket_name = 'city-planning-entitlements'

## Import data

In [3]:
census = pd.read_parquet(f's3://{bucket_name}/data/final/census_analysis_table.parquet')

In [4]:
crosswalk_parcels_tracts = pd.read_parquet(f's3://{bucket_name}/data/crosswalk_parcels_tracts.parquet')

# Find the number of parcels within tract (but drop duplicate geometries first)
subset_cols = ['num_AIN', 'parcelsqft', 'parcel_tot', 'TOC_Tier', 'GEOID']
no_dups = crosswalk_parcels_tracts.drop_duplicates(subset = subset_cols, keep = 'first')

num_AIN_by_tract = (no_dups.groupby(['GEOID'])
                    .agg({'AIN':'count'})
                    .reset_index()
                    .rename(columns = {'AIN': 'tot_parcels'})
                   )

crosswalk_parcels_tracts = pd.merge(crosswalk_parcels_tracts, num_AIN_by_tract, 
                                    on = 'GEOID', how = 'left', validate = 'm:1')

keep = ['AIN', 'tot_parcels', 'GEOID', 'pop']
crosswalk_parcels_tracts = crosswalk_parcels_tracts[keep]

## Make PCTS tract-level
* Aggregate and count the number of unique parent cases, number of parcels with ENT, and number of ENT with each suffix by tract

In [5]:
def make_tract_level(start_date):
    # Import data
    pcts = pd.read_parquet(f's3://{bucket_name}/data/final/master_pcts.parquet')
    parents = pd.read_parquet(f's3://{bucket_name}/data/final/parents_with_suffix.parquet')
    
    # Subset PCTS, we'll start with last 10 years
    pcts = pcts[pcts.CASE_FILE_DATE >= start_date]
    
    # (1a) Only keep parent cases
    m1 = pd.merge(pcts, parents, on = 'PARENT_CASE', how = 'inner', validate = 'm:1')

    # (1b) Make cases parcel-level
    parents_by_parcel = (m1.groupby(['AIN'])
                         .agg({'PARENT_CASE':'count'})
                         .reset_index()
                         .rename(columns = {'PARENT_CASE':'num_cases'})
                        )

    # (1c) Merge in tract info and aggregate to tract-level
    m2 = pd.merge(parents_by_parcel, crosswalk_parcels_tracts, on = 'AIN', how = 'inner', validate = '1:1')

    # Even though the column num_AIN shows there are some parcels with more than 1 obs,
    # once we merged in parent cases, no AIN shows up more than once

    ent_by_tract = m2.groupby(['GEOID', 'pop']).agg({'num_cases':'sum', 'AIN':'count'}).reset_index()
    
    
    # (2a) Only keep suffixes
    suffix = m1.loc[:, '1A':'ZV']
    
    m3 = pd.merge(m1[['AIN']], suffix, left_index = True, right_index = True)

    # (2b) Make suffixes parcel-level
    suffix_by_parcel = (m3.pivot_table(index = 'AIN', aggfunc = 'sum')
                        .reset_index()
                       )

    # (2c) Merge in tract info and aggregate to tract-level
    m4 = pd.merge(suffix_by_parcel, crosswalk_parcels_tracts, 
                                 on = 'AIN', how = 'left', validate = '1:1')

    # Aggregate the number of suffixes by tract
    suffix_by_tract = m4.pivot_table(index = ['GEOID', 'pop'], aggfunc = 'sum').reset_index()
    
    
    # (3) Merge number of cases and suffixes by tract
    df = pd.merge(ent_by_tract, suffix_by_tract, on = ['GEOID', 'pop'], how = 'left', validate = '1:1')
        
    # (4) Make sure everything returns as integers and not floats
    colnames = list(df.columns)

    for r in ['GEOID', 'pop', 'AIN']:
        colnames.remove(r)
    
    # Count number of suffixes, since each ENT can have any number of suffixes involved
    df[colnames] = df[colnames].fillna(0).astype(int) 
    df['num_suffix'] = df[colnames].sum(axis=1)
        
    return df

In [6]:
start_date = "2010-01"
df = make_tract_level(start_date)

In [7]:
df = pd.merge(df, census, on = 'GEOID', how = 'left', validate = '1:1')
df.head()

,GEOID,pop,num_cases,AIN,1A,2A,A,AC,ACI,ADD1,...,ZCJ,ZV,tot_parcels,num_suffix,p_renter,p_zero_veh,p_transit,p_nonwhite,p_low_income_total,p_low_income_nonwhite
0,06037101110,4566,20,10,0,0,0,0,0,0,...,0,2,11640,11690,0.521214,0.000000,0.053970,0.231803,0.496241,0.459930
1,06037101122,3064,11,7,0,0,0,0,0,0,...,0,0,9212,9235,0.178417,0.006817,0.009963,0.124112,0.184713,0.186567
2,06037101210,6043,68,36,0,0,0,0,0,0,...,0,1,18828,18976,0.876399,0.089531,0.135740,0.211666,0.677725,0.560870
3,06037101220,3340,20,14,2,0,0,0,0,0,...,0,0,9674,9717,0.603374,0.083278,0.083278,0.286817,0.649923,0.485175
4,06037101300,4285,35,17,1,0,0,1,0,0,...,0,0,23919,23995,0.083059,0.010289,0.088192,0.137771,0.303833,0.144654


In [8]:
df.to_parquet(f's3://{bucket_name}/data/final/pcts_with_census.parquet')

## Export a table to use in Tableau

In [9]:
df = pd.read_parquet(f's3://{bucket_name}/data/final/pcts_with_census.parquet')
tracts = catalog.census_tracts.read()

In [10]:
df = pd.merge(df, tracts[['GEOID10', 'geometry']], 
              left_on = 'GEOID', right_on = 'GEOID10', how = 'inner', validate = 'm:1')

In [11]:
df = gpd.GeoDataFrame(df)
df.crs = {'init':'epsg:2229'}

/opt/conda/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [12]:
# Add quick summary stats
df = df.assign(
   cases_per_parcel = df.num_cases / df.tot_parcels,
   suffixes_per_parcel = df.num_suffix / df.tot_parcels,
)

In [13]:
df.to_crs({'init':'epsg:4326'}).to_file(driver = 'ESRI Shapefile', filename = '../gis/final/test_tableau')

/opt/conda/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [14]:
df.sort_values(['cases_per_parcel'], ascending = False)

,GEOID,pop,num_cases,AIN,1A,2A,A,AC,ACI,ADD1,...,p_renter,p_zero_veh,p_transit,p_nonwhite,p_low_income_total,p_low_income_nonwhite,GEOID10,geometry,cases_per_parcel,suffixes_per_parcel
972,06037980023,0,6,3,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,06037980023,"MULTIPOLYGON (((6368953.155 1909403.805, 63690...",0.666667,2.333333
531,06037207501,2249,13,3,0,0,0,0,0,0,...,0.811279,0.119809,0.494894,0.638988,0.400719,0.537060,06037207501,"MULTIPOLYGON (((6485613.978 1842580.430, 64855...",0.619048,2.523810
679,06037222700,4145,341,26,0,0,0,9,0,0,...,1.000000,2.082007,3.951040,0.520124,0.880342,1.000000,06037222700,"MULTIPOLYGON (((6475728.003 1831042.042, 64761...",0.504438,2.187870
970,06037980015,514,7,2,0,0,0,0,0,0,...,1.000000,0.058824,0.093426,0.712230,0.413043,0.387435,06037980015,"MULTIPOLYGON (((6467779.957 1740632.011, 64685...",0.437500,2.000000
966,06037702502,4171,6,3,0,0,0,0,0,0,...,0.359026,0.030395,0.229483,0.458448,0.240122,0.281806,06037702502,"MULTIPOLYGON (((6448597.497 1826929.250, 64485...",0.400000,1.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,06037109800,5494,1,1,0,0,0,0,0,0,...,0.229818,0.025262,0.130639,0.540571,0.288581,0.200658,06037109800,"MULTIPOLYGON (((6419006.533 1916361.846, 64189...",0.000788,1.001576
963,06037297500,5258,1483,1363,0,0,0,0,0,0,...,0.296703,0.008140,0.103876,0.230210,0.204577,0.173697,06037297500,"MULTIPOLYGON (((6469389.987 1721195.543, 64700...",0.000788,1.001577
341,06037143901,4146,747,527,13,8,0,0,0,0,...,0.094786,0.008599,0.074355,0.119272,0.139256,0.098592,06037143901,"MULTIPOLYGON (((6441818.559 1875683.163, 64418...",0.000781,1.001726
334,06037143400,6443,818,800,2,0,0,1,0,0,...,0.446322,0.037616,0.079873,0.163577,0.198803,0.216667,06037143400,"MULTIPOLYGON (((6441718.401 1879977.463, 64417...",0.000776,1.001559
